# The Message Passing Interface MPI

## Introduction  to MPI
The Message Passing Interface (MPI) is:

- Particularly useful for - but not limited to - distributed memory machines.
- The _de facto_ standard parallel programming interface.

Many implementations exist - MPICH, OpenMPI, ...

Interfaces in: 
- C/C++ 
- Fortran and ... 
- Python wrappers (mpi4py)

## What is message passing?
The program is launched as separate processes (tasks) each with their own address space.
- To achieve parallelism we could have each process work on different data 

Information must be **explicitly moved** from process to process:
- A task can access the data of another process through passing messages (a copy of the data is passed from one process to another)

Two main classes of message passing:
- **Point-to-point** operations, involving only two processes
- **Collective** operations, involving a group of processes

## mpi4py

mpi4py supports both point-to-point and collective communications for "generic" Python objects as well as "buffer-type" objects
- for generic Python objects, use the all-lowercase methods, e.g. send, recv, isend, irecv, bcast, scatter, gather, reduce
- for buffer-type objects (e.g. numpy arrays) use the uppercase versions, e.g. Send, ...
    

## Hello World!

In [ ]:
%%writefile hello_world_mpi.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

print('Hello from process {} out of {}'.format(rank, size))

## Communicators
A communicator is a group of processes that can talk to each other. 

- the size of the communicator is obtained by the `Get_size` method

Within that group each process is assigned a unique rank (assigned by the system when we launch the program).

- The rank of each process is retrieved via the `Get_rank` method 

In the above code, we defined set "comm" to the default communicator `MPI.COMM_WORLD`. 

## Executing an MPI code

There is nothing in the MPI standard that specifies how an MPI code should be executed. 

Typically, it will be launched with `mpirun` or `mpiexec` followed by python
e.g. `mpirun -n 4 python3 hello_world_python.py`

On Piz Daint we launch with `srun`:

In [ ]:
! srun -n 8 python hello_world_mpi.py

## Point-to-point communication

Point-to-point communication is sending message/data from one process to another. 

For point-to-point communication between Python objects, mpi4py provides the `send` and `recv` methods that are similar to those in MPI. 

In the following example, rank 0 passes a Python dictionary object to another processes (rank 1):


In [ ]:
%%writefile send_recv.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data = {'a': 9, 'b': 5.001}
    comm.send(data, dest=1, tag=42)
    print('Process {} sent data:'.format(rank), data)
    
elif rank == 1:
    data = comm.recv(source=0, tag=42)
    print('Process {} received data:'.format(rank), data)


In [ ]:
! srun -n 2 python send_recv.py

<div class="alert alert-warning alert-block alert-info"><b>Exercise:</b> Generalize the above example so that the master process sends the dictionary to an arbitrary number of processes. Use point-to-point communication.</div>

In [ ]:
%%writefile send_recv_for_loop.py
# write your solution here
# hint: add a for loop for the send
# hint: use the rank as the tag

In [ ]:
# try it out:
# if your code hangs/deadlocks, you might need to `killall srun` from the terminal.
! srun -n 6 python send_recv_for_loop.py

In [ ]:
# show a sample solution
%pycat send_recv_for_loop_solution.py

## Nonblocking point-to-point communication
In the above examples, the functions block the caller until the data buffers involved in the communication can be safely reused by the application program.

For potentially increased performance we can try to overlap communication and computation by using nonblocking send and receives.

`isend` and `irecv` are non-blocking methods that immediately return `Request` objects.

We can use the `wait` method to manage the completion of the communication. 

You can then perform computation between `comm.isend(...)` and `req.wait()`.

In [ ]:
%%writefile isend_irecv_for_loop.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data = {'a': 9, 'b': 5.001}
    for i in range(1, size):
        req = comm.isend(data, dest=i, tag=i)
        req.wait()
        print('Process {} sent data:'.format(rank), data)

else:
    req = comm.irecv(source=0, tag=rank)
    data = req.wait()
    print('Process {} received data:'.format(rank), data)

In [ ]:
! srun -n 12 python isend_irecv_for_loop.py

## Collective communication 
Generally groups of processes need to exchange messages between themselves. Rather than explicitly sending and receiving such messages from point to point, MPI comes with group operations known as collectives.
- Broadcast, scatter, gather and reduction
- Implementations can optimize performance

### Broadcast
Send from one process to all other processes.

![broadcast](03-broadcast.png) 

We can rewrite our above exmaple using a broadcast.

In [ ]:
%%writefile broadcast.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data = {'a': 9, 'b': 5.001}
           
else:
    data = None
data = comm.bcast(data, root=0)
print('Process {} received data:'.format(rank), data)

In [ ]:
! srun -n 12 python broadcast.py

### Scatter
Split data into chunks and send a chunk to individual processes to work on.

![scatter](03-scatter.png)

In [ ]:
%%writefile scatter.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None
data = comm.scatter(data, root=0)
print('Process {} received data:'.format(rank), data)
assert data == (rank+1)**2

In [ ]:
! srun -n 4 python scatter.py

### Gather
Gather the chunks and bring them to the root process

![gather](03-gather.png) 

In [ ]:
%%writefile gather.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

data = (rank+1)**2
data = comm.gather(data, root=0)
if rank == 0:
    for i in range(size):
        assert data[i] == (i+1)**2
else:
    assert data is None

In [ ]:
! srun -n 4 python gather.py

Note: In contrast, allgather will return the results to all processes.

### Reduction

Gather results, and then do some computation.

Examples of reductions are:
- MPI_SUM - Sums the elements. 
- MPI_PROD - Multiplies all elements. 
- MPI_MAX - Returns the maximum element 
- MPI_MIN - Returns the minimum element

![reduce](03-reduce.png)

In [ ]:
%%writefile reduction.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# element assigned to be the rank of the processor
value = np.array(rank,'d')

print(' Rank: ',rank, ' value = ', value)

# initialize the np arrays that will store the results:
value_sum      = np.array(0.0,'d')
value_max      = np.array(0.0,'d')

# perform the reductions:
comm.Reduce(value, value_sum, op=MPI.SUM, root=0)
comm.Reduce(value, value_max, op=MPI.MAX, root=0)

if rank == 0:
    print(' Rank 0: value_sum =    ',value_sum)
    print(' Rank 0: value_max =    ',value_max)

In [ ]:
! srun -n 10 python reduction.py

## Lower case vs. upper case versions
In Python there are two versions of the various MPI methods:

- lower case (send, recv, gather, etc.)
- Upper case (Send, Recv, Gather, etc.) 

You can transmit arbitrary Python data types using the lower-case version of the methods. mpi4py will serialize the data type, send it to the remote process, then deserialize it back to the original data type (a process known as "pickling" and "unpickling"). This can add significant overhead to the MPI operation.

For sending "buffer-like" objects, you can use the upper-case versions. The data object must support Python's "single-segment buffer interface". Examples of objects you can send this way are numpy arrays and strings.

Use the upper-case versions where possible as they will be faster (close to the speed of MPI communication in C).

- the memory of the receiving buffer needs to be allocated prior to the communication
- the size of the sending buffer should not exceed that of the receiving buffer 
- mpi4py expects the buffer-like objects to have contiguous memory (e.g. as is the case with numpy arrays) 


Here we pass a numpy array from the master node to the other processes:


In [ ]:
%%writefile uppercase.py
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
    data = np.arange(4.)
    # master process sends data to worker processes by
    # going through the ranks of all worker processes
    for i in range(1, size):
        comm.Send(data, dest=i, tag=i)
        print('Process {} sent data:'.format(rank), data)

else:
    # initialize the receiving buffer
    data = np.zeros(4)
    # receive data from master process
    comm.Recv(data, source=0, tag=rank)
    print('Process {} received data:'.format(rank), data)

In [ ]:
! srun -n 8 python uppercase.py

Note: 

- the data array has to exist at the time of the `Recv` call.
- the `Recv` method takes data as the first argument (in contrast to the `recv` method which returns the data object).

## Summary
- `mpi4py` is the most commonly used Python interface to MPI 
- MPI calls are via the communicator object
- You can communicate arbitrary Python objects
- NumPy arrays can be communicated with similar speed to C and Fortran

# Using mpi4py in a Notebook

So far we have been running our MPI code with `srun -n X ... python <code>`. 

To use MPI directly in a Jupyter Notebook you need to combine:

- mpi4py, and
- IPython Parallel 

## IPython Parallel 

In regular IPython we have a client (the frontend) and a kernel which executes the code. And they communciate with messages. 

So, as IPython already does remote execution... if you have _one_ remote kernel, why not have _one hundred_?

These are called IPython Parallel "engines"

<div>
<img src="ipyparallel.png" style="width:300px"/>
</div>
Rather than having clients (blue) connect directly to kernels (green) as in notebook, you have an intermediary of a hub (with schedulers) - known as the "controller". The client communicates only with the controller. The controller keeps track of the available engines and forwards requests from the client to the engines. It schedules the work and monitors its status. The results are communicated through the controller back to the client.

To use IPython for parallel computing, you need to start one instance of the controller and one or more instances of the engines. The controller and each engine can run on different machines or on the same machine.

There are three ways to start the controller and engines:

- Separately, using the **ipcontroller** and **ipengine** commands.
- In an automated manner using the **ipcluster** command.
- From a custom **magic** developed inhouse `import ipcmagic`

We will use the first method, which is "manual", but provides the most transparency.

<div class="alert alert-block alert-info">
    <b>Note:</b> The following commands need to be entered in a terminal. File > New > Terminal. A terminal will open as a new tab. Grab the tab and pull it to the right to have the terminal next to your notebook.
</div>

```
$ ipcontroller start &
$ srun -n 4 ipengine --mpi 
```    

The IPython Parallel engines need to be started using the `mpirun` command (or equivalent). On our system:

- Start the **ipcontroller** 
- Start the **ipengines** using `srun` and with `--mpi` argument.


Now let's see how we access our "Cluster". [IPython][IP] comes with a module [ipyparallel][IPp] that is used to access the engines, we just started. We first need to import Client.

[IPp]: https://ipyparallel.readthedocs.io/en/latest/
[IP]: http://www.ipython.org

The client is started by first importing it from ipyparallel and then by initalizing it. 

In [ ]:
import ipyparallel as ipp

<div class="alert alert-block alert-danger">
    <b>Note:</b> If you receive an error ModuleNotFoundError: No module named 'ipyparallel', ensure that you have the miniconda-pythonhpc kernel loaded
</div>


In [ ]:
rc = ipp.Client(profile="default")

List the ids of the engines attached:

In [ ]:
rc.ids

## Parallel Magics

IPython makes it very easy to use IPyParallel. It provides the magic commands ``%px`` and ``%%px`` to execute code in parallel. The target attribute is used to pick the engines, you want. By default, all the engines of the last Client object created are used. 

In [ ]:
%%px --target 0:2
import os, socket
print(os.getpid())
print(socket.gethostname())

In [ ]:
%%px
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = MPI.COMM_WORLD.Get_rank()
size = MPI.COMM_WORLD.Get_size()

A = np.zeros((size,size))
if rank==0:
    A = np.random.randn(size, size)
    print("Original array on root process\n", A)
local_a = np.zeros(size)

comm.Scatter(A, local_a, root=0)
print("Process", rank, "received", local_a)